In [3]:
#a
import requests
from bs4 import BeautifulSoup

url = "https://www.24h.com.vn/gia-vang-hom-nay-c425.html"
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

table = soup.find("table", class_="gia-vang-search-data-table")

rows = table.find_all("tr")
print(table)

<table class="gia-vang-search-data-table">
<tbody>
<tr class="" data-seach="sjc" onclick="active_gia_vang_theo_ma('sjc');">
<td class="colorGrey pdL10" width="213px">
<h2>SJC</h2>
</td>
<td class="colorGrey" width="136px"><span class="fixW">97,100</span>
<img src="https://cdn.24h.com.vn/images/2014/ttcb/btn_down2019.png"/>
<span class="colorRed">1700</span>
</td>
<td class="colorGrey" width="154px">
<span class="fixW">100,100</span>
<img src="https://cdn.24h.com.vn/images/2014/ttcb/btn_down2019.png"/>
<span class="colorRed">1200</span>
</td>
<td class="colorBlur" width="100px">98,800</td>
<td class="colorBlur">101,300</td>
</tr>
<tr class="" data-seach="doji_hn" onclick="active_gia_vang_theo_ma('doji_hn');">
<td class="colorGrey pdL10" width="213px">
<h2>DOJI HN</h2>
</td>
<td class="colorGrey" width="136px"><span class="fixW">97,100</span>
<img src="https://cdn.24h.com.vn/images/2014/ttcb/btn_down2019.png"/>
<span class="colorRed">1700</span>
</td>
<td class="colorGrey" width="154px">

In [4]:
#b
for row in rows:
    gold_brand = row.find("h2")
    if gold_brand:
        gold_brand = gold_brand.get_text(strip=True)
        cols = row.find_all("td")
        if len(cols) > 4:
            today_buy = cols[1].get_text(strip=True).replace(",", "")
            today_sell = cols[2].get_text(strip=True).replace(",", "")
            yesterday_buy = cols[3].get_text(strip=True).replace(",", "")
            yesterday_sell = cols[4].get_text(strip=True).replace(",", "")

            # In ra dữ liệu đã làm sạch
            print(f"Thương hiệu vàng: {gold_brand}")
            print(f"Giá mua hôm nay: {today_buy} VND")
            print(f"Giá bán hôm nay: {today_sell} VND")
            print(f"Giá mua hôm qua: {yesterday_buy} VND")
            print(f"Giá bán hôm qua: {yesterday_sell} VND")
            print("-" * 40)

Thương hiệu vàng: SJC
Giá mua hôm nay: 971001700 VND
Giá bán hôm nay: 1001001200 VND
Giá mua hôm qua: 98800 VND
Giá bán hôm qua: 101300 VND
----------------------------------------
Thương hiệu vàng: DOJI HN
Giá mua hôm nay: 971001700 VND
Giá bán hôm nay: 1001001200 VND
Giá mua hôm qua: 98800 VND
Giá bán hôm qua: 101300 VND
----------------------------------------
Thương hiệu vàng: DOJI SG
Giá mua hôm nay: 971001700 VND
Giá bán hôm nay: 1001001200 VND
Giá mua hôm qua: 98800 VND
Giá bán hôm qua: 101300 VND
----------------------------------------
Thương hiệu vàng: BTMC SJC
Giá mua hôm nay: 972001600 VND
Giá bán hôm nay: 1001001200 VND
Giá mua hôm qua: 98800 VND
Giá bán hôm qua: 101300 VND
----------------------------------------
Thương hiệu vàng: Phú Qúy SJC
Giá mua hôm nay: 971001500 VND
Giá bán hôm nay: 1001001200 VND
Giá mua hôm qua: 98600 VND
Giá bán hôm qua: 101300 VND
----------------------------------------
Thương hiệu vàng: PNJ TP.HCM
Giá mua hôm nay: 975001200 VND
Giá bán hôm na

In [6]:
#c
import requests
from bs4 import BeautifulSoup
import psycopg2
from datetime import datetime
DB_CONFIG = {
    "port": 5432,
    "user": "airflow",
    "password": "airflow",
    "database": "airflow"
}


def create_table(cur):
    cur.execute("""
        CREATE TABLE IF NOT EXISTS gold_prices (
            id SERIAL PRIMARY KEY,
            timestamp TIMESTAMP,
            brand TEXT,
            today_buy NUMERIC,
            today_sell NUMERIC,
            yesterday_buy NUMERIC,
            yesterday_sell NUMERIC
        );
    """)

# -------------------------------------
# Hàm lưu dữ liệu vào PostgreSQL
# -------------------------------------
def insert_data(cur, data):
    cur.execute("""
        INSERT INTO gold_prices (timestamp, brand, today_buy, today_sell, yesterday_buy, yesterday_sell)
        VALUES (%s, %s, %s, %s, %s, %s);
    """, data)

# -------------------------------------
# Crawl và lưu dữ liệu
# -------------------------------------
def crawl_and_store():
    url = "https://www.24h.com.vn/gia-vang-hom-nay-c425.html"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.find("table", class_="gia-vang-search-data-table")
    rows = table.find_all("tr")
    timestamp = datetime.now()

    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    create_table(cur)

    for row in rows:
        gold_brand = row.find("h2")
        if gold_brand:
            brand = gold_brand.text.strip()
            try:
                cols = row.find_all("td")
                today_buy = float(cols[1].text.strip().split("\n")[0].replace(",", ""))
                today_sell = float(cols[2].text.strip().split("\n")[0].replace(",", ""))
                yesterday_buy = float(cols[3].text.strip().replace(",", ""))
                yesterday_sell = float(cols[4].text.strip().replace(",", ""))

                insert_data(cur, (timestamp, brand, today_buy, today_sell, yesterday_buy, yesterday_sell))

                print(f"✅ Lưu: {brand}")
            except Exception as e:
                print(f"⚠️ Lỗi khi xử lý {brand}: {e}")

    conn.commit()
    cur.close()
    conn.close()
    print("✅ Đã lưu toàn bộ dữ liệu vào PostgreSQL.")

if __name__ == "__main__":
    crawl_and_store()

ModuleNotFoundError: No module named 'psycopg2'